In [1]:
import spark_setup
spark_setup.setup_pyspark_env()
import spark_utils

In [2]:
%%time
sc = spark_utils.get_spark_context()

Ambari - http://10.0.1.21:8080
All Applications - http://10.0.1.23:8088/cluster
CPU times: user 24 ms, sys: 4 ms, total: 28 ms
Wall time: 16.2 s


In [4]:
import pandas as pd
from pyspark.sql import SparkSession

ss = SparkSession(sc)

In [5]:
from hdfs import InsecureClient
hdfs_client = InsecureClient("http://cluster1:50070", user='hdfs')

# Load data to HDFS

In [13]:
import time

def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        print '%r (%r, %r) %2.2f sec' % \
              (method.__name__, args, kw, te-ts)
        return result
    return timed

In [12]:
hdfs_client.delete("/task1", recursive=True)

True

In [14]:
%%time
import subprocess

@timeit
def unzip_to_hdfs(fn):
    fn_out = fn.replace(".zip", "")
    print subprocess.check_output("unzip -p /data/{0} | hadoop fs -put - /task1/{1}".format(fn, fn_out), shell=True)
    
fns = [
    "clicks_test.csv.zip",
    "clicks_train.csv.zip",
    "documents_categories.csv.zip",
    "documents_entities.csv.zip",
    "documents_meta.csv.zip",
    "documents_topics.csv.zip",
    "events.csv.zip",
    "page_views.csv.zip",
    "page_views_sample.csv.zip",
    "promoted_content.csv.zip",
    "sample_submission.csv.zip"
]

for fn in fns:
    unzip_to_hdfs(fn)


'unzip_to_hdfs' (('clicks_test.csv.zip',), {}) 7.43 sec

'unzip_to_hdfs' (('clicks_train.csv.zip',), {}) 17.91 sec

'unzip_to_hdfs' (('documents_categories.csv.zip',), {}) 10.08 sec

'unzip_to_hdfs' (('documents_entities.csv.zip',), {}) 5.79 sec

'unzip_to_hdfs' (('documents_meta.csv.zip',), {}) 2.81 sec

'unzip_to_hdfs' (('documents_topics.csv.zip',), {}) 5.74 sec

'unzip_to_hdfs' (('events.csv.zip',), {}) 25.10 sec

'unzip_to_hdfs' (('page_views.csv.zip',), {}) 1274.26 sec

'unzip_to_hdfs' (('page_views_sample.csv.zip',), {}) 6.40 sec

'unzip_to_hdfs' (('promoted_content.csv.zip',), {}) 2.25 sec

'unzip_to_hdfs' (('sample_submission.csv.zip',), {}) 5.54 sec
CPU times: user 168 ms, sys: 60 ms, total: 228 ms
Wall time: 22min 43s


In [32]:
! hadoop fs -du -s -h /task1

92.9 G  /task1


In [ ]:
# files are written on cluster1 node only, need to balance HDFS on cluster

In [16]:
! hdfs dfsadmin -setBalancerBandwidth 1000000000

Balancer bandwidth is set to 1000000000


In [23]:
! hdfs balancer -threshold 5 > balancer.log 2>&1

# Read example

In [26]:
pvdf = ss.read.csv("/task1/page_views.csv", header=True)

In [27]:
pvdf.dtypes

[('uuid', 'string'),
 ('document_id', 'string'),
 ('timestamp', 'string'),
 ('platform', 'string'),
 ('geo_location', 'string'),
 ('traffic_source', 'string')]

In [30]:
pvdf.show(5)

+--------------+-----------+---------+--------+------------+--------------+
|          uuid|document_id|timestamp|platform|geo_location|traffic_source|
+--------------+-----------+---------+--------+------------+--------------+
|1fd5f051fba643|        120| 31905835|       1|          RS|             2|
|8557aa9004be3b|        120| 32053104|       1|       VN>44|             2|
|c351b277a358f0|        120| 54013023|       1|       KR>12|             1|
|8205775c5387f9|        120| 44196592|       1|       IN>16|             2|
|9cb0ccd8458371|        120| 65817371|       1|   US>CA>807|             2|
+--------------+-----------+---------+--------+------------+--------------+
only showing top 5 rows



In [31]:
%%time
pvdf.count()

CPU times: user 56 ms, sys: 8 ms, total: 64 ms
Wall time: 7min 23s


2034275448